In [1]:
import wget
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from tqdm import tqdm

# dr3

## 下载

In [2]:
wget.download('https://cloud.datacentral.org.au/teamdata/GALAH/public/GALAH_DR3/GALAH_DR3_main_allstar_v2.fits', r"./")

100% [......................................................................] 873518400 / 873518400

'F:\\My_trial\\Fits_preprocessed\\other_catalog/GALAH_DR3_main_allstar_v2.fits'

## 处理

### 将星表按照需要，存到df

In [10]:
# 读取数据
hdu = fits.open(r"./GALAH_DR3_main_allstar_v2.fits")
# hdu[1].header

In [11]:
# 将需要的目录存到df

lable_list = ['ra_dr2', 'dec_dr2','flag_sp','snr_c3_iraf','star_id','sobject_id','teff','logg','fe_h','Mg_fe','Si_fe','K_fe','Ca_fe','Ti_fe','Cr_fe','Ni_fe','C_fe','O_fe' ,'Al_fe','Mn_fe','Na_fe','V_fe']

df_GALAH = pd.DataFrame(data=None, columns=lable_list)
for lable_name in tqdm(lable_list):
    df_GALAH[lable_name] = hdu[1].data[lable_name]
df_GALAH.info()

100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:01<00:00, 14.41it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588571 entries, 0 to 588570
Data columns (total 22 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ra_dr2       588571 non-null  >f8   
 1   dec_dr2      588571 non-null  >f8   
 2   flag_sp      588571 non-null  >i8   
 3   snr_c3_iraf  588571 non-null  >f8   
 4   star_id      588571 non-null  object
 5   sobject_id   588571 non-null  >i8   
 6   teff         581149 non-null  >f4   
 7   logg         581149 non-null  >f4   
 8   fe_h         567408 non-null  >f4   
 9   Mg_fe        504056 non-null  >f8   
 10  Si_fe        515027 non-null  >f8   
 11  K_fe         496920 non-null  >f8   
 12  Ca_fe        517490 non-null  >f8   
 13  Ti_fe        466507 non-null  >f8   
 14  Cr_fe        507932 non-null  >f8   
 15  Ni_fe        423662 non-null  >f8   
 16  C_fe         293907 non-null  >f8   
 17  O_fe         519276 non-null  >f8   
 18  Al_fe        522895 non-null  >f8   
 19  Mn

In [12]:
df_GALAH.shape

(588571, 22)

### 筛选掉不好的数据

In [5]:
# 将整个 DataFrame 或相关列转换为小端格式。这可以确保所有操作（如筛选、删除等）都在正确的数据类型上执行。
for col in df_GALAH.columns:
    if df_GALAH[col].dtype.byteorder == '>':
        df_GALAH[col] = df_GALAH[col].values.byteswap().newbyteorder()

# 按flag_sp筛选
df_GALAH =df_GALAH.drop(
    df_GALAH.index[np.where(df_GALAH['flag_sp']!=0)[0]])
df_GALAH.reset_index(drop=True, inplace=True)    # drop：是否删除原始索引列   inplace：是否在原始数据框上直接修改
df_GALAH.shape

# 按snr_c3_iraf<30筛选
df_GALAH =df_GALAH.drop(
    df_GALAH.index[np.where(df_GALAH['snr_c3_iraf']<30)[0]])
df_GALAH.reset_index(drop=True, inplace=True)    # drop：是否删除原始索引列   inplace：是否在原始数据框上直接修改
df_GALAH.shape



(326264, 20)

### 将X/Fe 转换为X/H

In [6]:
# X/Fe转换成X/H
# lable_list = ['ra_dr2', 'dec_dr2','flag_sp','snr_c3_iraf','star_id','sobject_id','teff','logg','fe_h','Mg_fe','Si_fe','K_fe','Ca_fe','Ti_fe','Cr_fe','Ni_fe','C_fe','O_fe' ,'Al_fe','Mn_fe']
output_label = ['GALAH_starid', 'GALAH_RA', 'GALAH_Dec', 'GALAH_Teff[K]','GALAH_Logg',
                'GALAH_FeH', 'GALAH_MgH', 'GALAH_SiH', 'GALAH_KH', 'GALAH_CaH',
                'GALAH_TiH', 'GALAH_CrH', 'GALAH_NiH', 'GALAH_CH', 'GALAH_OH',
                'GALAH_AlH', 'GALAH_MnH']

# 'teff','logg','fe_h','Mg_fe','Si_fe','K_fe','Ca_fe','Ti_fe','Cr_fe','Ni_fe','C_fe','O_fe' ,'Al_fe','Mn_fe'

# ['star_id','ra_dr2', 'dec_dr2','teff','logg','fe_h','Mg_fe','Si_fe','K_fe','Ca_fe','Ti_fe','Cr_fe','Ni_fe','C_fe','O_fe' ,'Al_fe','Mn_fe']
df_GALAH = pd.DataFrame(data=df_GALAH[['star_id','ra_dr2', 'dec_dr2','teff','logg','fe_h','Mg_fe','Si_fe','K_fe','Ca_fe','Ti_fe','Cr_fe','Ni_fe','C_fe','O_fe' ,'Al_fe','Mn_fe']].values,
                                columns=output_label)
df_GALAH = df_GALAH.iloc[1:]
for label_name in ['GALAH_MgH', 'GALAH_SiH', 'GALAH_KH', 'GALAH_CaH',
                'GALAH_TiH', 'GALAH_CrH', 'GALAH_NiH', 'GALAH_CH', 'GALAH_OH',
                'GALAH_AlH', 'GALAH_MnH']:
        df_GALAH[label_name] = df_GALAH[label_name] + df_GALAH['GALAH_FeH']
 
df_GALAH

,GALAH_starid,GALAH_RA,GALAH_Dec,GALAH_Teff[K],GALAH_Logg,GALAH_FeH,GALAH_MgH,GALAH_SiH,GALAH_KH,GALAH_CaH,GALAH_TiH,GALAH_CrH,GALAH_NiH,GALAH_CH,GALAH_OH,GALAH_AlH,GALAH_MnH
1,03405609-6844483,55.233798,-68.746746,5642.742676,4.298538,0.082866,0.020715,0.175627,-0.054782,-0.146646,0.125296,0.16298,0.073291,0.20428,0.154053,0.303096,0.009558
2,03403587-6844432,55.149569,-68.745502,4940.007324,4.590864,0.109674,0.162903,0.028643,-0.016138,0.108688,-0.079381,0.055596,0.117284,NaN,0.330039,0.203572,0.167831
3,03402113-6848514,55.087989,-68.814367,5789.730469,4.427538,-0.421067,-0.310231,-0.295358,-0.380096,-0.332973,0.092756,-0.409865,-0.360667,0.186586,0.157015,-0.27097,-0.492918
4,03394376-6848101,54.932398,-68.802856,4679.781738,1.944841,-1.203136,-0.968172,-1.060492,-1.304515,-1.002976,NaN,-1.372157,-1.481865,NaN,-0.611822,-0.970984,-1.428613
5,03405458-6854480,55.227613,-68.913237,6170.653809,4.146284,-0.384692,-0.233238,-0.318847,-0.540235,-0.361372,NaN,-0.303339,NaN,0.067389,-0.203362,-0.31586,-0.471397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326259,15142072-4415325,228.586315,-44.259058,4531.215332,2.410332,-0.293808,-0.144124,-0.146496,-0.339019,-0.11385,-0.143996,-0.385375,-0.273325,NaN,0.062936,0.08953,-0.480828
326260,15151322-4415182,228.805089,-44.255053,6059.499512,3.519315,-0.045074,-0.15768,-0.059793,0.349271,0.122638,-0.090163,-0.165575,-0.349195,-0.385639,0.147953,-0.17905,0.006529
326261,15162269-4414571,229.09451,-44.24921,4132.972656,0.959392,-1.003942,-0.916899,-0.91241,-1.199549,-0.652221,-0.837043,-1.168993,-1.137659,NaN,NaN,-1.100304,-1.477088
326262,15161847-4418028,229.076923,-44.300799,4776.927246,2.420797,0.007314,-0.022626,0.002215,-0.087745,0.175625,-0.026534,-0.020608,0.101908,NaN,0.203465,0.194115,0.003137


### 保存数据

In [7]:
df_GALAH.to_csv('df_GALAH.csv', index=False) 

In [ ]:
# 'C_fe' 'O_fe' 'Na_fe''Mg_fe''Al_fe''Si_fe''K_fe''Ca_fe''Sc_fe''Sc2_fe''Ti_fe''Ti2_fe''V_fe''Cr_fe''Cr2_fe''Mn_fe''Co_fe' 'Ni_fe''Cu_fe''Zn_fe''Rb_fe''Sr_fe''Y_fe''Zr_fe''Mo_fe''Ru_fe''Ba_fe''La_fe''Ce_fe''Nd_fe''Sm_fe''Eu_fe'

# flag_sp!=0  snr_c3_iraf<30

# dr 4

## 下载

In [4]:
wget.download('https://cloud.datacentral.org.au/teamdata/GALAH/public/GALAH_DR4/catalogs/galah_dr4_allstar_240705.fits', r"./")

100% [......................................................................] 757998720 / 757998720

'.//galah_dr4_allstar_240705.fits'

## 处理

### 将星表按照需要，存到df

In [2]:
# 读取数据
hdu = fits.open(r"./galah_dr4_allstar_240705.fits")
# hdu[1].header

In [3]:
# 将需要的目录存到df

lable_list = ['ra', 'dec', 'sobject_id', 'tmass_id', 'flag_sp', 'gaiadr3_source_id', 'snr_px_ccd3', 'teff', 'logg', 'rv_comp_1', 'rv_comp_2', 'fe_h', 'mg_fe', 'si_fe', 'k_fe', 'ca_fe', 'ti_fe', 'cr_fe', 'ni_fe', 'c_fe', 'o_fe' ,'al_fe', 'mn_fe', 'na_fe', 'v_fe']

df_GALAH = pd.DataFrame(data=None, columns=lable_list)
for lable_name in tqdm(lable_list):
    df_GALAH[lable_name] = hdu[1].data[lable_name]
df_GALAH.info()

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:01<00:00, 20.88it/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 917588 entries, 0 to 917587
Data columns (total 25 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ra                 917588 non-null  >f8   
 1   dec                917588 non-null  >f8   
 2   sobject_id         917588 non-null  >i8   
 3   tmass_id           917588 non-null  object
 4   flag_sp            917588 non-null  >i4   
 5   gaiadr3_source_id  917588 non-null  >i8   
 6   snr_px_ccd3        911958 non-null  >f4   
 7   teff               906688 non-null  >f4   
 8   logg               906688 non-null  >f4   
 9   rv_comp_1          906689 non-null  >f4   
 10  rv_comp_2          32566 non-null   >f4   
 11  fe_h               906688 non-null  >f4   
 12  mg_fe              898029 non-null  >f4   
 13  si_fe              885806 non-null  >f4   
 14  k_fe               860264 non-null  >f4   
 15  ca_fe              900125 non-null  >f4   
 16  ti_fe              9

In [4]:
df_GALAH.shape

(917588, 25)

### 筛选掉不好的数据

In [5]:
# 将整个 DataFrame 或相关列转换为小端格式。这可以确保所有操作（如筛选、删除等）都在正确的数据类型上执行。
for col in df_GALAH.columns:
    if df_GALAH[col].dtype.byteorder == '>':
        df_GALAH[col] = df_GALAH[col].values.byteswap().newbyteorder()

# 按flag_sp筛选
df_GALAH =df_GALAH.drop(
    df_GALAH.index[np.where(df_GALAH['flag_sp']!=0)[0]])
df_GALAH.reset_index(drop=True, inplace=True)    # drop：是否删除原始索引列   inplace：是否在原始数据框上直接修改
df_GALAH.shape

# # 按snr_c3_iraf<30筛选
# df_GALAH =df_GALAH.drop(
#     df_GALAH.index[np.where(df_GALAH['snr_px_ccd3']<30)[0]])
# df_GALAH.reset_index(drop=True, inplace=True)    # drop：是否删除原始索引列   inplace：是否在原始数据框上直接修改
# df_GALAH.shape



(663075, 25)

In [6]:
df_GALAH.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 663075 entries, 0 to 663074
Data columns (total 25 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ra                 663075 non-null  float64
 1   dec                663075 non-null  float64
 2   sobject_id         663075 non-null  int64  
 3   tmass_id           663075 non-null  object 
 4   flag_sp            663075 non-null  int32  
 5   gaiadr3_source_id  663075 non-null  int64  
 6   snr_px_ccd3        663075 non-null  float32
 7   teff               663075 non-null  float32
 8   logg               663075 non-null  float32
 9   rv_comp_1          663075 non-null  float32
 10  rv_comp_2          0 non-null       float32
 11  fe_h               663075 non-null  float32
 12  mg_fe              661885 non-null  float32
 13  si_fe              661016 non-null  float32
 14  k_fe               662135 non-null  float32
 15  ca_fe              662663 non-null  float32
 16  ti

### 将X/Fe 转换为X/H

In [7]:
# X/Fe转换成X/H
# lable_list = ['ra', 'dec', 'sobject_id', 'tmass_id', 'flag_sp', 'gaiadr3_source_id', 'snr_px_ccd3', 'teff', 'logg', 'rv_comp_1', 'rv_comp_2', 'fe_h', 'mg_fe', 'si_fe', 'k_fe', 'ca_fe', 'ti_fe', 'cr_fe', 'ni_fe', 'c_fe', 'o_fe' ,'al_fe', 'mn_fe', 'na_fe', 'v_fe']
output_label = ['GALAH_sobject_id', 'GALAH_RA', 'GALAH_Dec', 'GALAH_Teff[K]','GALAH_Logg', 'GALAH_RV',
                'GALAH_FeH', 'GALAH_MgH', 'GALAH_SiH', 'GALAH_KH', 'GALAH_CaH',
                'GALAH_TiH', 'GALAH_CrH', 'GALAH_NiH', 'GALAH_CH', 'GALAH_OH',
                'GALAH_AlH', 'GALAH_MnH', 'GALAH_NaH', 'GALAH_VH']

# 'teff','logg','fe_h','Mg_fe','Si_fe','K_fe','Ca_fe','Ti_fe','Cr_fe','Ni_fe','C_fe','O_fe' ,'Al_fe','Mn_fe'

# ['star_id','ra_dr2', 'dec_dr2','teff','logg','fe_h','Mg_fe','Si_fe','K_fe','Ca_fe','Ti_fe','Cr_fe','Ni_fe','C_fe','O_fe' ,'Al_fe','Mn_fe']
df_GALAH = pd.DataFrame(data=df_GALAH[['sobject_id','ra', 'dec','teff','logg', 'rv_comp_1', 'fe_h','mg_fe', 'si_fe', 'k_fe', 'ca_fe', 'ti_fe', 'cr_fe', 'ni_fe', 'c_fe', 'o_fe' ,'al_fe', 'mn_fe', 'na_fe', 'v_fe']].values,
                                columns=output_label)
df_GALAH = df_GALAH.iloc[1:]
for label_name in ['GALAH_MgH', 'GALAH_SiH', 'GALAH_KH', 'GALAH_CaH',
                'GALAH_TiH', 'GALAH_CrH', 'GALAH_NiH', 'GALAH_CH', 'GALAH_OH',
                'GALAH_AlH', 'GALAH_MnH', 'GALAH_NaH', 'GALAH_VH']:
        df_GALAH[label_name] = df_GALAH[label_name] + df_GALAH['GALAH_FeH']
 
df_GALAH

,GALAH_sobject_id,GALAH_RA,GALAH_Dec,GALAH_Teff[K],GALAH_Logg,GALAH_RV,GALAH_FeH,GALAH_MgH,GALAH_SiH,GALAH_KH,GALAH_CaH,GALAH_TiH,GALAH_CrH,GALAH_NiH,GALAH_CH,GALAH_OH,GALAH_AlH,GALAH_MnH,GALAH_NaH,GALAH_VH
1,1.611160e+14,0.001000,-57.162334,4900.511719,4.464450,-17.269478,-1.032346,-0.485245,-0.522659,-0.836103,-0.733011,-0.623313,-0.921852,-1.047303,-1.056093,-1.641717,-0.636157,-1.270972,-1.026297,-0.815148
2,1.611160e+14,0.001000,-57.713528,5543.346191,4.175371,35.673534,-0.654349,-0.339513,-0.499854,-0.412886,-0.442860,-0.477679,-0.656255,-0.640972,-0.365437,-0.047547,-0.628637,-0.852933,-0.618969,-0.506713
3,1.407130e+14,0.001042,-75.687943,5529.029297,4.521021,24.175571,0.135527,0.179893,0.122973,0.055832,0.102860,0.086971,0.150604,0.133135,0.130371,0.135678,0.232372,0.188496,0.076286,0.252581
4,1.407080e+14,0.001583,-77.940865,4977.451172,3.172151,51.569466,-0.612790,-0.312235,-0.321094,-0.533563,-0.381498,-0.319066,-0.602469,-0.551210,-0.469420,-0.269039,-0.037948,-0.734186,-0.550215,-0.316458
5,1.611180e+14,0.001667,-51.243973,5694.408691,4.450530,33.268894,0.107221,0.176302,0.131125,0.039020,0.088876,0.068954,0.079446,0.134381,0.119479,0.059279,0.157909,0.171137,0.094745,0.157260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663070,1.611160e+14,359.995819,-56.851055,5481.137695,4.445670,-6.889793,0.103741,0.221993,0.150510,0.164297,0.131254,0.091621,0.108528,0.149705,0.206520,0.101541,0.351872,0.181172,0.186800,0.199482
663071,2.008010e+14,359.997070,-54.356224,5428.497070,4.455420,15.372953,0.010281,0.092273,0.094932,-0.052973,-0.033793,0.015348,0.027000,0.056165,0.147511,0.175296,0.206904,-0.013190,-0.031190,0.140210
663072,1.707110e+14,359.998779,-63.245998,5902.784180,4.012203,24.662785,-0.415962,-0.334707,-0.383137,-0.270259,-0.377403,-0.346356,-0.329139,-0.402470,-0.405404,-0.067628,-0.571348,-0.529947,-0.324590,-0.339275
663073,2.008010e+14,0.000167,-52.963306,5399.747559,4.333981,3.227795,-0.095362,0.054439,-0.024124,0.215236,0.004322,-0.097859,-0.089264,-0.050772,0.005421,-0.147737,0.212130,-0.046102,-0.014221,0.065007


In [8]:
df_GALAH.to_csv('df_GALAH_dr4.csv', index=False) 